In [2]:
import os
import sys
base_dir = os.path.join("/home/jovyan/work/Corona/COVID-19")
sys.path.append(base_dir)
import model as md
import pandas as pd

In [3]:
#NewsExtractor(2)
def createJson(date, day, json=False):
    news_reader = md.NewsReader(data_dir="data", month=date, date="{}-{}".format(date, day), string_filter=True)
    repartition = news_reader.repartition[
        (news_reader.repartition["h1"]=="Az aktív fertőzöttek megyei eloszlása")
        & (news_reader.repartition["date"].dt.day == int(day))
    ]["body"].unique()
    date_of_info = news_reader.repartition[
        (news_reader.repartition["h1"]=="Az aktív fertőzöttek megyei eloszlása")
        & (news_reader.repartition["date"].dt.day == int(day))
    ]["date"].unique()
    print(date_of_info)
    if len(repartition)>0:
        table = "Aktív fertőzöttek száma{}".format(repartition[0].split("Aktív fertőzöttek száma")[1]).split("\n\n\n\n\n")
        df = pd.DataFrame(table, columns=["full"])
        df["full"] = df["full"].str.split("\n\n\n")
        df["len"] = df["full"].apply(lambda cell: len(cell))
        df = df[df["len"]==3].copy()
        df['region'], df['active'], df['death'] = zip(*df["full"])
        df['date'] = "{}-{}".format(date, day)
        df[['date', 'region', 'active', 'death']].to_json('repartition/{}-{}.json'.format(date, day), orient='records')
        return df[['date', 'region', 'active', 'death']]

In [10]:
createJson("2020-11", "28", True)

INFO:root:2020-11-28 08:58:00
INFO:root:https://koronavirus.gov.hu/cikkek/6268-fovel-emelkedett-beazonositott-fertozottek-szama-es-elhunyt-152-beteg
INFO:root:
6268 újabb magyar állampolgárnál mutatták ki az új koronavírus-fertőzést (COVID-19), ezzel 204 708 főre nőtt a hazánkban beazonosított fertőzöttek száma. Elhunyt 152 többségében idős, krónikus beteg, így az elhunytak száma 4 516 főre emelkedett. A gyógyultak száma 54 021 főre emelkedett, az aktív fertőzöttek száma 146 171 fő. Az aktív fertőzöttek 20%-a, az elhunytak 24%-a, a gyógyultak 26%-a budapesti. 7536 koronavírusos beteget ápolnak kórházban, közülük 644-en vannak lélegeztetőgépen. A beazonosított fertőzöttek számában megjelennek a mentők által, a szűrőbuszokon, és a kórházi dolgozók körében gyorsteszteléssel kiszűrtek is. A bölcsődei, óvodai, iskolai és szociális dolgozók körében végzett célzott csoportos tesztelés összesített eredménye jövő hét elején várható.

Egész Európában, a szomszédos országokban és Magyarországon i